In [4]:
import pandas as pd 

def readGeo(filename:str):
    f = open(filename, 'r')
    l = f.readline()
    sampleTitles = []
    geoAccessions = []
    sampleDescriptions = []
    while('!series_matrix_table_begin' not in l):
        l = l.replace('"', '').strip()
        if('!Sample_title' in l):
            sampleTitles = l.split('\t')[1:]
        if('!Sample_geo_accession' in l):
            geoAccessions = l.split('\t')[1:]
        if('!Sample_description' in l):
            sampleDescriptions = l.split('\t')[1:]
        l = f.readline()
    groups = {}
    for i in range(0, len(sampleTitles)):
        st = sampleTitles[i][:-2]
        if st not in groups:
            groups[st] = [geoAccessions[i]]
        else:
            groups[st].append(geoAccessions[i])
    df = pd.read_csv(f, sep='\t', comment='!')
    return groups, df


In [6]:
g, df = readGeo('GSE56133_series_matrix_s.txt')
print(g)
df

{'WT Untreated': ['GSM1356325', 'GSM1356326', 'GSM1356327'], 'WT Amp': ['GSM1356328', 'GSM1356329', 'GSM1356330'], 'WT Gent': ['GSM1356331', 'GSM1356332', 'GSM1356333'], 'WT Kan': ['GSM1356334', 'GSM1356335', 'GSM1356336'], 'WT Nor': ['GSM1356337', 'GSM1356338', 'GSM1356339'], 'WT H2O2': ['GSM1356340', 'GSM1356341', 'GSM1356342'], 'Fur KO Untreated': ['GSM1356343', 'GSM1356344', 'GSM1356345'], 'Fur KO Gent': ['GSM1356346', 'GSM1356347', 'GSM1356348'], 'Hpx KO Untreated': ['GSM1356349', 'GSM1356350', 'GSM1356351']}


,ID_REF,GSM1356325,GSM1356326,GSM1356327,GSM1356328,GSM1356329,GSM1356330,GSM1356331,GSM1356332,GSM1356333,...,GSM1356342,GSM1356343,GSM1356344,GSM1356345,GSM1356346,GSM1356347,GSM1356348,GSM1356349,GSM1356350,GSM1356351
0,1759068_at,4.995971,5.310001,5.317402,5.417772,5.198019,4.965284,4.860005,5.235138,5.444700,...,5.267795,4.940357,5.047132,4.663667,4.999695,5.135909,5.115006,5.034367,5.198019,5.154724
1,1759069_at,11.155949,11.227223,11.000524,9.870207,9.885231,9.830734,10.936557,11.001372,10.853160,...,9.938893,10.544297,10.474940,10.503799,10.285315,10.437576,10.304743,10.039345,9.979794,9.963714
2,1759070_s_at,8.459495,8.389346,8.297275,8.609185,8.590776,8.556534,8.362895,8.536375,8.491274,...,8.539582,8.221200,8.269841,8.215465,8.221200,8.366196,8.206101,8.300355,8.340707,8.389316
3,1759071_s_at,7.511163,7.009353,6.861405,6.646697,6.958215,6.816070,7.707205,7.157688,7.326739,...,6.488555,6.664908,6.856376,6.944385,6.874713,6.729841,6.918046,6.950605,6.962309,6.896834
4,1759072_s_at,4.913314,5.393651,5.446799,5.613249,5.236126,5.367812,5.367812,5.525210,5.718355,...,6.017708,4.770942,4.924177,4.516015,5.122281,5.365210,4.969870,5.388273,5.293297,5.279094
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2299,1761394_s_at,5.389093,6.238326,6.011162,6.284623,5.992039,6.041896,6.342665,5.795139,6.288796,...,6.146753,6.042261,5.855214,5.762748,6.143970,6.143970,6.083274,6.026182,6.168893,6.024530
2300,1761395_s_at,5.911529,5.422210,5.530839,6.578997,6.786730,7.014832,5.993140,6.224867,6.108655,...,6.933948,5.707370,5.613806,6.078961,6.562175,6.333287,6.366614,6.760414,6.936131,6.672079
2301,1761396_at,5.131057,5.520800,5.535203,5.716754,5.407864,5.463062,5.276455,5.362476,5.518466,...,5.513715,5.141412,5.119993,5.142079,5.362476,5.235136,5.189903,5.260606,5.526449,5.280491
2302,1761397_at,3.767377,3.603766,3.659480,3.902299,3.951266,3.721068,3.327233,3.597554,3.553185,...,3.785296,3.921125,3.485521,3.554161,3.323602,3.697986,3.721068,3.576256,3.508150,3.928423


In [13]:
import numpy as np
from scipy.stats.stats import ttest_ind

def stats(df:pd.DataFrame, groups:dict) -> pd.DataFrame:
    # [{}, {}, {},...]
    statsTable = []
    for index, row in df.iterrows():
        statsRow = {'ID_REF': row['ID_REF']}
        for gname, samples in groups.items():
            g = []
            for i in samples:
                g.append(float(row[i]))
            statsRow[gname] = np.mean(g)
        statsTable.append(statsRow)
        groupNames = list(groups.keys())
        pvalues = []
        for i in range(0, len(groupNames)):
            for j in range(i+1, len(groupNames)):
                g1 = []
                for k in groups[groupNames[i]]:
                    g1.append(float(row[k]))
                g2 = []
                for k in groups[groupNames[j]]:
                    g2.append(float(row[k]))
                t, p = ttest_ind(a=g1, b=g2, equal_var=False)
                pvalues.append(p)
        statsRow['p'] = min(pvalues)
    return pd.DataFrame(statsTable)

In [14]:
sdf = stats(df, g)
sdf

,ID_REF,WT Untreated,WT Amp,WT Gent,WT Kan,WT Nor,WT H2O2,Fur KO Untreated,Fur KO Gent,Hpx KO Untreated,p
0,1759068_at,5.207791,5.193692,5.179948,4.991450,5.708779,5.257199,4.883719,5.083537,5.129037,0.033998
1,1759069_at,11.127899,9.862057,10.930363,10.159795,10.922007,10.090836,10.507679,10.342545,9.994284,0.000022
2,1759070_s_at,8.382039,8.585498,8.463515,8.399711,8.313767,8.571304,8.235502,8.264499,8.343459,0.000121
3,1759071_s_at,7.127307,6.806994,7.397211,7.035167,5.801095,6.570106,6.821890,6.840867,6.936583,0.001538
4,1759072_s_at,5.251255,5.405729,5.537126,5.384023,6.050491,5.782192,4.737045,5.152454,5.320221,0.001268
...,...,...,...,...,...,...,...,...,...,...,...
2299,1761394_s_at,5.879527,6.106186,6.142200,6.104481,6.818898,6.124682,5.886741,6.123738,6.073202,0.000651
2300,1761395_s_at,5.621526,6.793520,6.108887,6.709910,5.761593,6.450757,5.800046,6.420692,6.789541,0.000617
2301,1761396_at,5.395687,5.529227,5.385799,5.410080,5.824484,5.361501,5.134495,5.262505,5.355849,0.003438
2302,1761397_at,3.676874,3.858211,3.492657,3.784175,3.923059,4.200926,3.653602,3.580885,3.670943,0.029936


In [15]:
sdf.sort_values('p')

,ID_REF,WT Untreated,WT Amp,WT Gent,WT Kan,WT Nor,WT H2O2,Fur KO Untreated,Fur KO Gent,Hpx KO Untreated,p
420,1759490_s_at,14.350693,14.410651,14.671015,14.506159,13.132491,14.301458,14.298237,14.465778,14.226040,7.607839e-09
2260,1761353_at,8.902147,9.369423,10.008917,9.728056,8.633407,9.266216,8.986585,9.190987,9.290921,2.249395e-08
703,1759774_at,8.370324,8.960422,7.303668,7.794790,10.085663,9.362510,13.229281,12.993194,11.514534,2.887134e-08
270,1759340_s_at,10.335354,11.883433,11.190363,11.869611,10.127936,11.200272,10.907364,11.447896,11.533444,3.388610e-08
729,1759801_s_at,13.715611,13.726065,13.936861,13.505349,13.658381,13.679332,14.582949,14.561827,13.951874,7.023843e-08
...,...,...,...,...,...,...,...,...,...,...,...
1046,1760122_s_at,3.444791,3.283224,3.467134,3.579948,3.560768,3.420097,3.424446,3.570515,3.759004,1.430708e-01
1143,1760220_s_at,3.802694,3.091072,3.188254,3.059371,3.068263,3.157253,3.800901,3.194623,3.138425,1.442984e-01
1025,1760101_at,3.236928,3.408694,3.348698,3.406254,3.358836,3.299442,3.229375,3.274083,3.438457,1.447705e-01
835,1759908_s_at,5.359168,5.278543,5.135869,5.317932,5.192194,5.192112,5.607812,5.304673,5.309222,1.761839e-01


In [16]:
def bonferroni(df:pd.DataFrame):
    geneCount = len(df) + 1 # 0 to 1-based
    bonfCorr = []
    for index, row in df.iterrows():
        corrP = row['p'] * geneCount
        if corrP > 1:
            corrP = 1
        bonfCorr.append(corrP)
    df['bp'] = pd.array(bonfCorr)
    return df

In [17]:
bonferroni(sdf)

,ID_REF,WT Untreated,WT Amp,WT Gent,WT Kan,WT Nor,WT H2O2,Fur KO Untreated,Fur KO Gent,Hpx KO Untreated,p,bp
0,1759068_at,5.207791,5.193692,5.179948,4.991450,5.708779,5.257199,4.883719,5.083537,5.129037,0.033998,1.000000
1,1759069_at,11.127899,9.862057,10.930363,10.159795,10.922007,10.090836,10.507679,10.342545,9.994284,0.000022,0.050691
2,1759070_s_at,8.382039,8.585498,8.463515,8.399711,8.313767,8.571304,8.235502,8.264499,8.343459,0.000121,0.278361
3,1759071_s_at,7.127307,6.806994,7.397211,7.035167,5.801095,6.570106,6.821890,6.840867,6.936583,0.001538,1.000000
4,1759072_s_at,5.251255,5.405729,5.537126,5.384023,6.050491,5.782192,4.737045,5.152454,5.320221,0.001268,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
2299,1761394_s_at,5.879527,6.106186,6.142200,6.104481,6.818898,6.124682,5.886741,6.123738,6.073202,0.000651,1.000000
2300,1761395_s_at,5.621526,6.793520,6.108887,6.709910,5.761593,6.450757,5.800046,6.420692,6.789541,0.000617,1.000000
2301,1761396_at,5.395687,5.529227,5.385799,5.410080,5.824484,5.361501,5.134495,5.262505,5.355849,0.003438,1.000000
2302,1761397_at,3.676874,3.858211,3.492657,3.784175,3.923059,4.200926,3.653602,3.580885,3.670943,0.029936,1.000000
